In [1]:
import logging
import json
import os
import sys
from tqdm import tqdm

sys.path.append(os.getcwd())

from CHAMP_utils import *

setOpenAi(keyid = 3)

GPT_MODEL = "gpt-3.5-turbo"


file_path = '/Users/natehu/Desktop/Tsinghua Research/Tsinghua TaDe/ToT复现/TOT/champ_116.json'
with open(file_path, 'r', encoding='utf-8') as file:
    dataset = json.load(file)

success_Q = 0
unsuccess_Q = 0
error_Q = 0
question_ids = range(len(dataset))

question_id = 0

question = dataset[question_id]['problem_text']
type = split_string_by_(dataset[question_id]['problem_identifier'])
gold_answer = dataset[question_id]['problem_answer']


decompose_steps = decompose_sql(question, type, GPT_MODEL)
steps, steps_dict = convert_steps_to_format(decompose_steps)
num_steps = len(steps)


solution = []

sys_q = f"""There is a math problem. I need you to solve it and give an answer.
Here is the problem:\n{question}

I have broken this math problem down into a series of smaller problems. I will assign you sub-problems one by one, and provide the results of the previous sub-problems as a reference for your reasoning.
Please solve the problem and respond according to mathematical logic.
"""  

for i in range(num_steps):
    subtask = steps[i]
    subask = f"""\nThe sub-problem to solve now is: {subtask}
Based on the information above, please provide a concise and clear answer"""
    
    if len(solution) == 0:
        query = subask
        Q = [{'role':'system', 'content':sys_q},
                {'role':'user', 'content':query}]
    else:
        answersSoFar = f"""\nSo far, the answers to the preceding sub-problems are as follows: The format is Sub-problem-Id: xxx; Sub-problem: xxx; Answer: xxx."""
        for index, value in enumerate(solution):
            answersSoFar += f"""\nSub-problem-Id: {index}; Sub-problem: {steps[index]}; Answer: {value}."""
        query = answersSoFar + subask
        Q = [{'role':'system', 'content':sys_q},
                {'role':'user', 'content':query}]

    result = askChatGPT(Q, model=GPT_MODEL, temperature=1)
    solution.append(result)

user_q = f"""There is a math problem:\n{question}

I have broken this math problem down into a series of smaller problems and each sub-problem is solved.
The sub-problems and their corresponding answers are as follows. (Format: Sub-problem-Id: xxx; Sub-problem: xxx; Answer: xxx.)"""

for index, value in enumerate(solution):
    user_q += f"""\nSub-problem-Id: {index}; Sub-problem: {steps[index]}; Answer: {value}."""

Q = [{'role':'system', 'content':sys_q},
        {'role':'user', 'content':user_q},
        {'role':'user', 'content':f"""Now that all the sub-problems have been solved, what is the final answer?
Please give the final answer without any additional explanation or clarification."""}]

finalResult = askChatGPT(Q, model=GPT_MODEL, temperature=1)
print("THE FINAL RESULT IS: ", finalResult)

judgeAnswer = {'role':'user', 'content':f"""Here is a math problem with a standard answer and a student's solution. Please help me determine if the student's solution is correct.
Problem: {question}

Standard answer: {gold_answer}

Answer: {finalResult}

If the student's answer is correct, just output True; otherwise, just output False.
No explanation is required.
"""}

Q_judge = [judgeAnswer]
ifcorrect = askChatGPT(Q_judge, model=GPT_MODEL, temperature=1)


if 'True' in ifcorrect:
    success_Q += 1

elif 'False' in ifcorrect:
    unsuccess_Q += 1

else:
    error_Q += 1



    


THE FINAL RESULT IS:  The final answer is:

\((n^2/2) \times (n^2/2 - n) \times (n^2/2 - 2n) \times ... \times (n^2/2 - (n-1)n)\)


In [2]:
solution

['The first rook can be placed in \\(n^2/2\\) ways without conflicting with any other rooks.',
 'Sub-problem-Id: 1; Sub-problem: How many ways can the second rook be placed on the chess board without conflicting with any other rooks?; Answer: After the first rook is placed, there will be \\(n^2/2 - n\\) squares left in the same row and column as the first rook. Therefore, the second rook can be placed in \\(n^2/2 - n\\) ways without conflicting with any other rooks.',
 'Given that the first rook is placed, there are \\(n^2/2 - n\\) squares left in the same row and column as the first rook.\nAfter the second rook is placed, there will be \\((n^2/2 - n) - n\\) squares left in the same rows and columns as the first two rooks for the third rook to be placed without conflicts.\nTherefore, the third rook can be placed in \\((n^2/2 - n) - n = n^2/2 - 2n\\) ways without conflicting with any other rooks.',
 'Sub-problem-Id: 3; Sub-problem: How many ways can the fourth rook be placed on the ches

In [3]:
Q

[{'role': 'system',
  'content': 'There is a math problem. I need you to solve it and give an answer.\nHere is the problem:\nOn a chess board, two rooks are placed peacefully if they are not on the same row or column. For an n x n chess board, find the number of ways that n rooks can be placed peacefully (i.e., any two are placed peacefully) and the placing is also invariant to a 180-degree rotation.\n\nI have broken this math problem down into a series of smaller problems. I will assign you sub-problems one by one, and provide the results of the previous sub-problems as a reference for your reasoning.\nPlease solve the problem and respond according to mathematical logic.\n'},
 {'role': 'user',
  'content': 'There is a math problem:\nOn a chess board, two rooks are placed peacefully if they are not on the same row or column. For an n x n chess board, find the number of ways that n rooks can be placed peacefully (i.e., any two are placed peacefully) and the placing is also invariant to 